# Combining Python, SQL DBs and AWS

## Aim: Create a RDB instance on AWS, connect to it through python, and execute queries.  


## Create an instance on AWS  
    
https://aws.amazon.com/getting-started/tutorials/create-mysql-db/

##  Using MYSQLWorkbench

Before we complicate things with using Python, we want to use MYSQLWorkbench to make sure we set up our AWS server correctly and can connect to it.  

- Create a connection to the AWS server
- Execute a simple create table query
- Drop the table


# Using Python with MYSQL DB

In [17]:
## importing 'mysql.connector' 
import mysql.connector

In [18]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector .connect(
    host = "localhost",
    user = "root",
    passwd = "dbms"
)

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost' (61)

### Documentation:

https://dev.mysql.com/doc/connector-python/en/connector-python-connectargs.html

## Using a config File:

You do not want to make your credentials viewable to everyone who might see this file.  So instead of explicity stating your credentials, we want to import them from another file.  

In [23]:
import config2

In [24]:
#config.pw_example

In [26]:
## Connecting to the database

## connecting to the database using 'connect()' method
## it takes 3 required parameters 'host', 'user', 'passwd'
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    password = config2.passwd
)

print(cnx) # it will print a connection object if everything is fine


In [27]:
cursor = cnx.cursor()

## Creating DB

In [28]:
import mysql.connector
from mysql.connector import errorcode

In [29]:
db_name = 'employees'

In [30]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database employees does not exists.
Database employees created successfully.


### Creating tables

In [31]:
DB_NAME = 'employees'

TABLES = {}
TABLES['employees'] = (
    "CREATE TABLE employees ("
    "  emp_no int(11) NOT NULL AUTO_INCREMENT,"
    "  birth_date date NOT NULL,"
    "  first_name varchar(14) NOT NULL,"
    "  last_name varchar(16) NOT NULL,"
    "  gender enum('M','F') NOT NULL,"
    "  hire_date date NOT NULL,"
    "  PRIMARY KEY (emp_no)"
    ") ENGINE=InnoDB")

TABLES['departments'] = (
    "CREATE TABLE departments ("
    "  dept_no char(4) NOT NULL,"
    "  dept_name varchar(40) NOT NULL,"
    "  PRIMARY KEY (dept_no), UNIQUE KEY dept_name (dept_name)"
    ") ENGINE=InnoDB")

TABLES['salaries'] = (
    "CREATE TABLE salaries ("
    "  emp_no int(11) NOT NULL,"
    "  salary int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT salaries_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_emp'] = (
    "CREATE TABLE dept_emp ("
    "  emp_no int(11) NOT NULL,"
    "  dept_no char(4) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no), KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_emp_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_emp_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['dept_manager'] = (
    "  CREATE TABLE dept_manager ("
    "  dept_no char(4) NOT NULL,"
    "  emp_no int(11) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date NOT NULL,"
    "  PRIMARY KEY (emp_no,dept_no),"
    "  KEY emp_no (emp_no),"
    "  KEY dept_no (dept_no),"
    "  CONSTRAINT dept_manager_ibfk_1 FOREIGN KEY (emp_no) "
    "     REFERENCES employees (emp_no) ON DELETE CASCADE,"
    "  CONSTRAINT dept_manager_ibfk_2 FOREIGN KEY (dept_no) "
    "     REFERENCES departments (dept_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

TABLES['titles'] = (
    "CREATE TABLE titles ("
    "  emp_no int(11) NOT NULL,"
    "  title varchar(50) NOT NULL,"
    "  from_date date NOT NULL,"
    "  to_date date DEFAULT NULL,"
    "  PRIMARY KEY (emp_no,title,from_date), KEY emp_no (emp_no),"
    "  CONSTRAINT titles_ibfk_1 FOREIGN KEY (emp_no)"
    "     REFERENCES employees (emp_no) ON DELETE CASCADE"
    ") ENGINE=InnoDB")

In [32]:
TABLES['employees']

"CREATE TABLE employees (  emp_no int(11) NOT NULL AUTO_INCREMENT,  birth_date date NOT NULL,  first_name varchar(14) NOT NULL,  last_name varchar(16) NOT NULL,  gender enum('M','F') NOT NULL,  hire_date date NOT NULL,  PRIMARY KEY (emp_no)) ENGINE=InnoDB"

In [34]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [35]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table employees: OK
Creating table departments: OK
Creating table salaries: OK
Creating table dept_emp: OK
Creating table dept_manager: OK
Creating table titles: OK


## Inserting Data

In [36]:
from datetime import date, datetime, timedelta

#since we closed the connection, we need to reinstatiate it
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

tomorrow = datetime.now().date() + timedelta(days=1)

add_employee = ("INSERT INTO employees "
               "(first_name, last_name, hire_date, gender, birth_date) "
               "VALUES (%s, %s, %s, %s, %s)")
add_salary = ("INSERT INTO salaries "
              "(emp_no, salary, from_date, to_date) "
              "VALUES (%(emp_no)s, %(salary)s, %(from_date)s, %(to_date)s)")

data_employee = ('Geert', 'Vanderkelen', tomorrow, 'M', date(1977, 6, 14))



In [37]:
# Insert new employee
cursor.execute(add_employee, data_employee)
emp_no = cursor.lastrowid

# Insert salary information
data_salary = {
  'emp_no': emp_no,
  'salary': 50000,
  'from_date': tomorrow,
  'to_date': date(9999, 1, 1),
}
cursor.execute(add_salary, data_salary)

# Make sure data is committed to the database
cnx.commit()

cursor.close()
cnx.close()

## Insert Many

cursor.executemany(operation, seq_of_params)

In [38]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [39]:
data = [
  ('Jane', date(2005, 2, 12)),
  ('Joe', date(2006, 5, 23)),
  ('John', date(2010, 10, 3)),
]
stmt = "INSERT INTO employees (first_name, hire_date) VALUES (%s, %s)"
cursor.executemany(stmt, data)

## Quyerying the DB

In [40]:
cnx = mysql.connector.connect(
    host = config2.host,
    user = config2.user,
    passwd = config2.passwd,
    database = DB_NAME
)
cursor = cnx.cursor()

In [42]:

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime(1999, 1, 1)
hire_end = datetime(1999, 12, 31)

cursor.execute(query, (hire_start, hire_end))

for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()


# Picking up from yesterday

## Select Statements

Select statements is the primary type of query you will write, as this is how we write q query to pull data back from a db


https://www.codecademy.com/articles/sql-commands

### SELECT
SELECT chooses the fields that you want displayed in your chart. This is the specific piece of information that you want to pull from your database.

**Syntax**

`SELECT first_name, last_name;`

Select * will pull back every column in the table


 ### FROM
FROM pinpoints the table that you want to pull the data from. 

**Syntax** 
`SELECT first_name, last_name
 FROM table;`

### WHERE
WHERE allows you to filter your query to be more specific. You  set up a conditional to filter your data

**syntax**

`SELECT first_name, last_name
FROM table
WHERE column = value`

### AND
AND allows you to add additional criteria to your WHERE statement. 


**Syntax**

`SELECT
     first_name, last_name
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value;`


### ORDER BY

Your ORDER BY clause will allow you to sort by any of the fields that you have specified in the SELECT statement. 

**Syntax**

`SELECT
     first_name, last_name
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
ORDER BY
     column;`






### GROUP BY
"GROUP BY" is similar to the function in pandas where it will aggregate data that has similarities.

Here is your SQL query:

**Syntax**

`SELECT
     first_name, COUNT(last_name)
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
GROUP BY
     column;`




### LIMIT
Depending on the amount of data you have in your database, it may take a long time to run the queries. It can be frustrating if you find yourself waiting a long time to run a query that you didn't really want to begin with. If you want to test our query, the LIMIT function is a great one to use because it allows you to limit the number of results you get.

**Syntax**

`SELECT
     first_name, COUNT(last_name)
FROM
     table
WHERE
     column = value
AND
     column BETWEEN value AND value
GROUP BY
     column
LIMIT
     100;`

## Applied: Working with our student data

### Import our student data

In [43]:
import json

f=open('students.json','r')
data=json.load(f)

### Create a connection and cursor to AWS db

In [56]:
# 
cnx = mysql.connector.connect(
        host = config2.host,
        user = config2.user,
        passwd = config2.passwd,
        database= DB_NAME)
cursor = cnx.cursor()

### Create a table for our student info

In [57]:

create_query = """
CREATE TABLE class_info (
name TEXT NOT NULL,
dob TEXT,
siblings REAL,
birthplace TEXT,
yearinnyc REAL,
favoritefood TEXT
);
"""

In [58]:
cursor.execute(create_query)

### Insert the student data into the table

In [47]:
#
keys_list = []
for student in data:
    print(student['name'])
    keys = student.keys()
    for word in keys:
        if word not in keys_list:
            keys_list.append(word)
print (keys_list)

Amora Sun
Alec Ryman
Catherine Wolk
Christopher Park
Eric Landstein
Jaden
James Moody
John Americk Canque
Justin Tennenbaum
Karolina Jozefowicz
Kathrin Verestoun
Kevin Sun
Kyle Baranko
mark cleverley
Matt Oliver
Michelle Venables
Paul E. Kruger
Quan Nguyen
Rebecca
Reuben Kavalov
Sarah Smith
Taylor Appel
['name', 'DOB', 'siblings', 'Birthplace', 'yearinnyc', 'favoritefood', 'birthdate', 'yearsinnyc', 'Favoritefood', 'birthplace']


In [50]:
students_list = []
    
for student in data:
    if 'DOB' in student.keys():
        student['birthdate'] = student['DOB']
    if 'Birthplace' in student.keys():
        student['birthplace'] = student.pop('Birthplace')
    if 'Favoritefood'in student.keys():
        student['favoritefood'] =student['Favoritefood']
    if 'yearinnyc' in student.keys():
        student['yearsinnyc'] = student['yearinnyc']
    student_tuple= (student['name'], student['birthdate'],
                   student['siblings'], student['birthplace'],
                   student['yearsinnyc'], student['favoritefood'])
    students_list.append(student_tuple)

In [51]:
students_list

[('Amora Sun', '11/24/1991', 'none', 'China', '22', 'Japanese'),
 ('Alec Ryman', '2-14-1993', 1, 'New York', 5, 'sushi'),
 ('Catherine Wolk', None, 2, 'Danbury, CT', 6, 'Banh Mi'),
 ('Christopher Park', '10-26-91', 2, 'New York City, New York', 10, 'shwarma'),
 ('Eric Landstein', '01-29-91', 1, 'Long Island, NY', 3, 'Pizza'),
 ('Jaden', '08-20-1996', 1, 'Hong Kong', 22, 'Pizza'),
 ('James Moody', '02-11-1991', 2, 'London, UK', 4, 'Vegan Pizza'),
 ('John Americk Canque',
  'October 8, 1987',
  'None',
  'Manila, Philippines',
  12,
  'Steak Chimichanga'),
 ('Justin Tennenbaum', '02-21-1995', 2, 'New Brunswick, NJ', 24, 'Ribs'),
 ('Karolina Jozefowicz', '05-05-1993', 1, 'Olsztyn, Poland', 0, 'bagels'),
 ('Kathrin Verestoun', '03-14-1988', 1, 'Odessa, Ukraine', 23, 'French Fries'),
 ('Kevin Sun', '11/6/97', 1, 'New Brunswick', 0, 'Mac and Cheese'),
 ('Kyle Baranko', '09-22-95', 2, 'Tucson, AZ', 1, 'Pad Thai'),
 ('mark cleverley', 'september 9, 1997', 0, 'london', 4, 'fish sticks'),
 ('Mat

In [60]:
#stmt = '''INSERT INTO class_info (name, dob, siblings, birthplace, yearinnyc, favoritefood) VALUES (%s, %s, %s, %s, %s, %s)'''
#cursor.executemany(stmt,students_list)
#cnx.commit()

In [11]:
import mysql.connector
import pandas as pd
import config2
DB_NAME = 'employees'

In [188]:
 cnx = mysql.connector.connect(
        host = config2.host,
        user = config2.user,
        passwd = config2.passwd,
        database= DB_NAME)
cursor = cnx.cursor()

In [189]:
cursor.execute('''SELECT name, dob FROM class_info''')

In [190]:
cursor.fetchall()

[('Amora Sun', '11/24/1991'),
 ('Alec Ryman', '2-14-1993'),
 ('Catherine Wolk', None),
 ('Christopher Park', '10-26-91'),
 ('Eric Landstein', '01-29-91'),
 ('Jaden', '08-20-1996'),
 ('James Moody', '02-11-1991'),
 ('John Americk Canque', 'October 8, 1987'),
 ('Justin Tennenbaum', '02-21-1995'),
 ('Karolina Jozefowicz', '05-05-1993'),
 ('Kathrin Verestoun', '03-14-1988'),
 ('Kevin Sun', '11/6/97'),
 ('Kyle Baranko', '09-22-95'),
 ('mark cleverley', 'september 9, 1997'),
 ('Matt Oliver', '04/18/1996'),
 ('Michelle Venables', '10/28/1993'),
 ('Paul E. Kruger', 'January 4, 1976'),
 ('Quan Nguyen', '3/2/94'),
 ('Rebecca', 'May 10'),
 ('Reuben Kavalov', '06/25/1995'),
 ('Sarah Smith', '09/26/1989'),
 ('Taylor Appel', '03-26-93')]

In [27]:
df = pd.DataFrame(cursor.fetchall())
df.columns = [x[0] for x in cursor.description]

In [28]:
cursor.description

[('name', 252, None, None, None, None, 0, 4113),
 ('dob', 252, None, None, None, None, 1, 16)]

In [40]:
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 
          'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12,}

In [29]:
df.dropna(inplace=True)
df.loc[df['dob'].str.contains('/'),'dob'] = df[df['dob'].str.contains('/')]['dob'].map(lambda x: '-'.join(x.split('/')))

In [36]:
df.loc[df['dob'].str.contains(','),'dob'] = df[df['dob'].str.contains(',')]['dob'].map(lambda x: '-'.join(x.split(', ')))

In [88]:
df[df['dob'].str.contains(' ')]

,name,dob
7,John Americk Canque,October 8-1987
13,mark cleverley,september 9-1997
16,Paul E. Kruger,January 4-1976
18,Rebecca,May 10


In [101]:
df.reset_index(drop=True, inplace=True)

In [102]:
for x in range(5):
    print(df.dob[x])


11-24-1991
2-14-1993
10-26-91
01-29-91
08-20-1996


In [180]:
test_df = df[df['dob'].str.contains(' ')]

In [181]:
test_df

,name,dob
6,John Americk Canque,October 8-1987
12,mark cleverley,september 9-1997
15,Paul E. Kruger,January 4-1976
17,Rebecca,May 10


In [182]:
test_df.reset_index(drop=True, inplace=True)

In [177]:
def alter_date(data):
    data['real_dob'] = 0
    for x in range(len(data['dob'])):
        data['dob'].iloc[x] = data['dob'].iloc[x].title()
        y1, y2 = data['dob'].iloc[x].split(' ')
        y1 = str(months[y1])
        listy = [y1,y2]
        data['real_dob'].iloc[x] = '-'.join(listy)
    data.drop(['dob'], axis=1, inplace=True)
    return data

# df[df['dob'].str.contains(' ')].apply(lambda x: )

In [176]:
# def alter_date(data,column):
#     data['real_dob'] = 0
#     for x in range(len(data)):
#         data[column].iloc[x] = data[column].iloc[x].title()
#         y1, y2 = data[column].iloc[x].split(' ')
#         y1 = str(months[y1])
#         listy = [y1,y2]
#         data['real_dob'].iloc[x] = '-'.join(listy)
#     data.drop([column], axis=1, inplace=True)    
#     return data

In [179]:
alter_date(test_df)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,name,real_dob
0,John Americk Canque,10-8-1987
1,mark cleverley,9-9-1997
2,Paul E. Kruger,1-4-1976
3,Rebecca,5-10


In [160]:
if 'real_dob' in test_df.columns:
    print(True)

True


SyntaxError: invalid syntax (<ipython-input-50-6ce0f7e94958>, line 1)

In [39]:
df[df['dob'].str.contains(' ')]['dob'].map(lambda x: x)

7       October 8-1987
13    september 9-1997
16      January 4-1976
Name: dob, dtype: object

### Write queries to answer the following questions:

**Questions**
- Which student was born closest to the cohort's graduation date?
- Which student has the most siblings?
- How many students are only children?
- Which 3 students have lived in NYC the shortest amount of time?
- How many students are native New Yorkers?
- Do any two students have the same favorite food?

In [210]:
 cnx = mysql.connector.connect(
        host = config2.host,
        user = config2.user,
        passwd = config2.passwd,
        database= DB_NAME)
cursor = cnx.cursor()

In [211]:
cursor.execute('''SELECT * FROM class_info WHERE siblings IN (SELECT MAX(siblings) FROM class_info);''')

In [212]:
cursor.fetchall()

[('Michelle Venables', '10/28/1993', 3.0, 'New Jersey', 2.0, 'Cucumbers'),
 ('Reuben Kavalov', '06/25/1995', 3.0, 'Queens, NYC', 10.0, 'Chili'),
 ('Taylor Appel', '03-26-93', 3.0, 'Morristown, NJ', 0.0, 'Mac n Cheese')]

In [213]:
cursor.execute('''SELECT * FROM class_info ORDER BY yearinnyc LIMIT 3;''')

In [214]:
cursor.fetchall()

[('Karolina Jozefowicz', '05-05-1993', 1.0, 'Olsztyn, Poland', 0.0, 'bagels'),
 ('Kevin Sun', '11/6/97', 1.0, 'New Brunswick', 0.0, 'Mac and Cheese'),
 ('Matt Oliver', '04/18/1996', 1.0, 'Topsham, Maine', 0.0, 'Reuben Sandwich')]